# 📗 Transformación de Archivos Raw a CSV Limpios
En este notebook convertimos los archivos `.txt` crudos descargados a archivos `.csv` estructurados y validamos su contenido.

## 📦 1. Cargar librerías necesarias

In [ ]:
import os
import pandas as pd

## 📁 2. Definir rutas y nombres de columnas
Se configuran los directorios y se establecen los nombres de las columnas según la documentación del dataset.

In [ ]:
raw_dir = 'data/raw'
clean_dir = 'data/cleaned'
os.makedirs(clean_dir, exist_ok=True)

columns = ['unit_number', 'time_in_cicles'] + [f'operational_setting{i}' for i in range(1, 4)] + [f'sensor_measurement{e}' for e in range(1, 22)]

## 🔄 3. Convertir archivos `.txt` a `.csv`
Si el archivo comienza con `RUL`, se trata de un archivo de vida útil restante. Si no, es un dataset de sensores.

In [ ]:
for file in os.listdir(raw_dir):
    if file.endswith('.txt') and file != 'readme.txt':
        file_path = os.path.join(raw_dir, file)
        if file.startswith('RUL'):
            fname = file.rstrip('.txt')
            df = pd.read_csv(file_path, header=None, names=['rul'], delim_whitespace=True)
            output_path = os.path.join(clean_dir, f'{fname}.csv')
            df.to_csv(output_path, index=False)
        else:
            fname = file.rstrip('.txt')
            df = pd.read_csv(file_path, header=None, names=columns, delim_whitespace=True)
            output_path = os.path.join(clean_dir, f'{fname}.csv')
            df.to_csv(output_path, index=False)
        print(f"✅ {file} ➡️ {fname}.csv guardado en {output_path}")

## ✅ 4. Validar columnas numéricas
Asegurarse de que todas las columnas sean numéricas. Si hay errores, los reporta con ejemplos.

In [ ]:
for file in os.listdir(clean_dir):
    file_path = os.path.join(clean_dir, file)
    try:
        if not file.startswith('rul'):
            df = pd.read_csv(file_path)
            for col in df.columns:
                if df[col].dtype == 'object':
                    try:
                        pd.to_numeric(df[col], errors='raise')
                    except Exception as e:
                        print(f"❌ Error en columna '{col}' del archivo '{file}': {e}")
                        print("👀 Valores problemáticos:")
                        print(df[col].unique()[:10])
    except Exception as e:
        print(f"💥 Error cargando {file}: {e}")